In [ ]:
import re
import requests
import mercantile
from rasterio.io import MemoryFile
from rasterio.features import sieve
from rasterio import features
from shapely.geometry import shape, mapping, Polygon
from shapely.ops import cascaded_union
from shapely.ops import transform
import numpy as np
from affine import Affine
from pyproj import Transformer
import json
import matplotlib.pyplot as plt
from descartes.patch import PolygonPatch
from scipy.ndimage.filters import gaussian_filter
from ipywidgets import interact, interactive, FloatSlider, interact_manual, HTML
import warnings
warnings.simplefilter("ignore")


style = """
    <style>
        .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
    </style>
    """
display(HTML(style))

%matplotlib inline

In [ ]:
print("Enter URL: E.g. https://mapproxy.osm.ch/tiles/AGIS2019/EPSG900913/{zoom}/{x}/{y}.png?origin=nw")
print("")
url = input()
print("")
print("Enter Zoom level: E.g. 12")
print("")
zoom = int(input())
print("")
print("Enter extent: (Format: min_lon,min_lat,max_lon,max_lat) E.g. 7.66646,47.08713,8.45118,47.63340")
print("")
bbox_str = input()

min_lon, min_lat, max_lon, max_lat = map(float, bbox_str.split(','))

print("Extent:")
print("West: {}".format(min_lon))
print("East: {}".format(max_lon))
print("South: {}".format(min_lat))
print("North: {}".format(max_lat))

parameters = {}
# {z} instead of {zoom}
if '{z}' in url:
    print('{z} found instead of {zoom} in tile url')
    exit()

if '{apikey}' in url:
    print("Not possible to check URL, apikey is required.")
    exit

if "{switch:" in url:
    match = re.search(r'switch:?([^}]*)', url)
    switches = match.group(1).split(',')
    tms_url = url.replace(match.group(0), 'switch')
    parameters['switch'] = switches[0]


def process_tile(tile):
    query_url = url
    if '{-y}' in url:
        y = 2 ** tile.z - 1 - tile.y
        query_url = query_url.replace('{-y}', str(y))
    elif '{!y}' in url:
        y = 2 ** (tile.z - 1) - 1 - tile.y
        query_url = query_url.replace('{!y}', str(y))
    else:
        query_url = query_url.replace('{y}', str(tile.y))
    parameters['x'] = tile.x
    parameters['zoom'] = tile.z
    query_url = query_url.format(**parameters)
    print("Request tile url:", query_url)

    data = None
    for i in range(3):
        try:
            r = requests.get(query_url)
            data = r.content
        except Exception as e:
            print(str(e))
        break
    if data is None:
        return []

    bounds = mercantile.bounds(*tile)

    try:
        with MemoryFile(data) as memfile:
            with memfile.open() as dataset:
                pixel_x = (bounds.east - bounds.west) / dataset.width
                pixel_y = (bounds.north - bounds.south) / dataset.height
                geotransform = (bounds.west, pixel_x, 0, bounds.north, 0, -pixel_y)

                data = np.zeros(shape=(dataset.height, dataset.width))
                for band in range(dataset.count):
                    data += dataset.read(band + 1)

                # Convert extrema to 0
                data[data < 2] = 0
                data[data > 254] = 0

                # Blur data to avoid holes
                data = gaussian_filter(data, sigma=3)

                # Convert other pixels to 1
                data[data > 0] = 1

                # Filter small areas
                data = sieve(data.astype(np.uint8),
                             size=int(dataset.height * dataset.width * 0.001),
                             connectivity=8)

                # Create shapes for all areas with value = 1
                shapes = list(features.shapes(data.astype(np.uint8), transform=Affine.from_gdal(*geotransform)))
                buffer_dist = max(pixel_x, pixel_y) * 5
                geoms = [shape(s[0]).buffer(buffer_dist).buffer(-buffer_dist)
                         for s in shapes if int(s[1]) == 1]
                return geoms
    except Exception as e:
        print(str(e))
    return []


def simplify(geom, distance=200):
    """ Simplify geometry in epsg:3857"""
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
    transformer_back = Transformer.from_crs("epsg:3857", "epsg:4326")
    geom_3857 = transform(transformer.transform, geom)
    geom_3857_simplified = geom_3857.simplify(distance, preserve_topology=False)
    geom_simplified = transform(transformer_back.transform, geom_3857_simplified)
    return geom_simplified


geoms = []
tiles = list(mercantile.tiles(west=min_lon, south=min_lat, east=max_lon, north=max_lat, zooms=zoom))
for tile in tiles:
    tile_geoms = process_tile(tile)
    if len(tile_geoms) > 0:
        geoms.extend(tile_geoms)
geom = cascaded_union(geoms)
geom = simplify(geom)


def plot_geometry(geom):
    bounds = geom.bounds
    fig, ax = plt.subplots()
    if isinstance(geom, Polygon):
        geom = [geom]
    for g in geom:
        patch = PolygonPatch(g, facecolor='#6699cc', edgecolor='#6699cc',
                             alpha=0.5, zorder=2)
        ax.add_patch(patch)
    ax.set_xlim(bounds[0], bounds[2])
    ax.set_ylim(bounds[1], bounds[3])
    plt.show()

plot_geometry(geom)

print(json.dumps(mapping(geom), indent=4))
